# ROC
---
_**R**ank **O**rder **C**entroid_

In [1]:
import numpy as np
import pandas as pd

In [2]:
import warnings
warnings.filterwarnings("ignore")

## 1. Introduction
---
The **ROC** (_Rank Order Centroid_) method is a simple and intuitive technique for defining criteria weights in multi-criteria decision problems. It is very useful when the decision maker does not know the degree of weighting of each criterion, but knows their order of importance. Therefore, the weights are calculated based on this ordering and distinct in terms of contribution (none of them will have the same weight).

### 1.1. Applications
---
The _ROC method_ can be used in a variety of situations where ease of understanding are relevant, such as:

- Supplier selection
- Assessment of job candidates
- Choice of products
- Feature priorization

For example, let's consider a set of elements with 5 random criteria $\{C_1, C_2, C_3, C_4, C_5\}$:

In [3]:
np.random.seed(0)
df = pd.DataFrame({
    "C1": np.random.randint(1, 10, 10),
    "C2": np.random.randint(1, 50, 10),
    "C3": np.random.randint(1, 20, 10),
    "C4": np.random.randint(1, 5, 10),
    "C5": np.random.randint(1, 8, 10)
})

df.style.background_gradient()

,C1,C2,C3,C4,C5
0,6,7,18,3,1
1,1,25,6,4,2
2,4,25,14,1,2
3,4,13,9,2,2
4,8,2,10,4,1
5,4,39,17,2,3
6,6,40,6,4,5
7,3,24,16,4,4
8,5,47,16,3,7
9,8,25,1,4,4


## 2. Method steps
---

### 2.1. Order of importance
---
The first step is to classify the criteria in **order of importance**.

In [4]:
ORDER = {
    # {criteria: order of importance}
    "C1": 3,
    "C2": 5,
    "C3": 2,
    "C4": 1,
    "C5": 4
}

### 2.2. Criteria weights
---
The second step is to **calculate the weight** of each criterion using the formula:

$$ \large
w_o = \frac{1}{m} \sum_{k=o}^m \frac{1}{k}
$$

In [5]:
m = len(ORDER)
K = np.arange(m) + 1

WEIGHTS = pd.Series({
    k: 1/m*(1/K[o-1:]).sum()
    for k, o in ORDER.items()
})

WEIGHTS

C1    0.156667
C2    0.040000
C3    0.256667
C4    0.456667
C5    0.090000
dtype: float64

### 2.3. Application
---
The _ROC method_ is supposed to be used together with other methods such as [PROMETHEE](./promethee.ipynb) ans others. As an example, we are going to calculate a kind of **score** for each data point, based on the weights applied for their criteria.

$$ \large
S_i = \sum_{k=1}^m w_k \cdot \frac{C_{ik}}{max(C_k)}
$$

Having that, we are able to define a **ranking** by ordering the score in descending way. 

In [6]:
df_ = df[list(ORDER)]
df_ = df_/df_.max()

df["score"] = np.sum(WEIGHTS*df_, axis=1)
df["ranking"] = (
    df["score"]
    .rank(ascending=False, method="first")
    .astype(int)
)

(
    df.sort_values("ranking")
    .style.background_gradient(
        subset=list(ORDER)
    )
)

,C1,C2,C3,C4,C5,score,ranking
7,3,24,16,4,4,0.815419,1
8,5,47,16,3,7,0.798565,2
4,8,2,10,4,1,0.770485,3
6,6,40,6,4,5,0.758050,4
0,6,7,18,3,1,0.735481,5
9,8,25,1,4,4,0.700298,6
5,4,39,17,2,3,0.620837,7
1,1,25,6,4,2,0.608796,8
3,4,13,9,2,2,0.471778,9
2,4,25,14,1,2,0.439121,10


## 3. Examples
---

### 3.1. Device priorization
---
As an example, let's priorize devices based on their attributes. The data was taken from [dxomark](https://www.dxomark.com/smartphones/) and we will consider 5 criteria: **price**, **camera**, **audio**, **display** and **battery**. The _price_ is a value we want to **minimize** so we have to prepare this feature before calculating the score. The other criteria are scores and we will maximize as usual.

In [7]:
df_ex1 = pd.DataFrame({
    "device": [
        "Apple iPhone 15 Pro Max", 
        "Apple iPhone 15 Pro",
        "Google Pixel 8 Pro",
        "Google Pixel 7 Pro",
        "Huawei Mate 50 Pro",
        "Huawei P50 Pro",
        "Samsung Galaxy S24 Ultra",
        "Samsung Galaxy S23 Ultra",
        "Xiaomi Mi 11 Ultra",
        "Xiaomi 13 Ultra",
    ],
    # criteria we want to minimize
    "price": [1199, 999, 999, 899, 1299, 907, 1299, 1199, 1200, 908],
    # criteria we want to maximize
    "camera": [154, 154, 153, 147, 149, 143, 144, 140, 141, 140],
    "audio": [143, 142, 142, 137, 144, 119, 139, 139, 119, 140],
    "display": [151, 151, 154, 148, 133, 134, 155, 148, 125, 130],
    "battery": [134, 114, 111, 102, 103, 123, 130, 142, 108, 133]
})

df_ex1.style.background_gradient()

,device,price,camera,audio,display,battery
0,Apple iPhone 15 Pro Max,1199,154,143,151,134
1,Apple iPhone 15 Pro,999,154,142,151,114
2,Google Pixel 8 Pro,999,153,142,154,111
3,Google Pixel 7 Pro,899,147,137,148,102
4,Huawei Mate 50 Pro,1299,149,144,133,103
5,Huawei P50 Pro,907,143,119,134,123
6,Samsung Galaxy S24 Ultra,1299,144,139,155,130
7,Samsung Galaxy S23 Ultra,1199,140,139,148,142
8,Xiaomi Mi 11 Ultra,1200,141,119,125,108
9,Xiaomi 13 Ultra,908,140,140,130,133


In [8]:
# Criteria order of importance
ORDER_ex1 = {
    "price": 1,
    "camera": 2,
    "audio": 5,
    "display": 4,
    "battery": 3
}

# Criteria weights method
m = len(ORDER_ex1)
K = np.arange(m) + 1

WEIGHTS_ex1 = pd.Series({
    k: 1/m*(1/K[o-1:]).sum()
    for k, o in ORDER_ex1.items()
})

# Data preparation and price minimization
df_ex1_ = df_ex1[list(ORDER_ex1)]
df_ex1_ = df_ex1_/df_ex1_.max()
df_ex1_["price"] = 1 - df_ex1_["price"]

# Score and Ranking
df_ex1["score"] = np.sum(WEIGHTS_ex1*df_ex1_, axis=1)
df_ex1["ranking"] = (
    df_ex1["score"]
    .rank(ascending=False, method="first")
    .astype(int)
)

# Output
(
    df_ex1.sort_values("ranking")
    .style.background_gradient(
        subset=list(ORDER_ex1)
    )
)

,device,price,camera,audio,display,battery,score,ranking
9,Xiaomi 13 Ultra,908,140,140,130,133,0.631900,1
5,Huawei P50 Pro,907,143,119,134,123,0.622708,2
3,Google Pixel 7 Pro,899,147,137,148,102,0.622147,3
1,Apple iPhone 15 Pro,999,154,142,151,114,0.615029,4
2,Google Pixel 8 Pro,999,153,142,154,111,0.611794,5
0,Apple iPhone 15 Pro Max,1199,154,143,151,134,0.567062,6
7,Samsung Galaxy S23 Ultra,1199,140,139,148,142,0.549702,7
6,Samsung Galaxy S24 Ultra,1299,144,139,155,130,0.512038,8
8,Xiaomi Mi 11 Ultra,1200,141,119,125,108,0.494595,9
4,Huawei Mate 50 Pro,1299,149,144,133,103,0.479198,10
